In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Segmentation ##
**In digital image processing and computer vision, image segmentation is the process of partitioning a digital image into multiple segments (sets of pixels, also known as image objects). The goal of segmentation is to simplify and/or change the representation of an image into something that is more meaningful and easier to analyze. Image segmentation is typically used to locate objects and boundaries (lines, curves, etc.) in images. More precisely, image segmentation is the process of assigning a label to every pixel in an image such that pixels with the same label share certain characteristics.**

**In this notebook we will try to implement Image Segmentation using Deep Learning.
  We will train a U-net model.
  "https://towardsdatascience.com/understanding-semantic-segmentation-with-unet-6be4f42d4b47",
  on Camvid Dataset. you can find this dataset easily on kaggle. I have taken it from     https://www.kaggle.com/carlolepelaars/camvid**


**Importing Libraries**

In [ ]:
import matplotlib.pyplot as plt
import keras
from tensorflow.keras import Sequential 
from tensorflow.keras.utils import Sequence, to_categorical, plot_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense, Input, MaxPooling2D, concatenate, BatchNormalization, Activation, Dropout
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from random import sample, choice
from PIL import Image

import warnings
warnings.filterwarnings("ignore")

**The CAMVID dataset consists of training(data and labels), and validation images(both data and label) and test data. so, we will make lists of directories of Train images and Val images which we will use to load the images later on whenever required.**

In [ ]:
train_img_lst = os.listdir("../input/camvid/CamVid/train")
val_img_lst = os.listdir("../input/camvid/CamVid/val")
test_img_lst = os.listdir("../input/camvid/CamVid/test")
print(len(train_img_lst),len(val_img_lst), len(test_img_lst))
print(type(train_img_lst[0].split('.')[0]))

**So, We have 369 images in Training data(we are using lesser than actual CamVid Data) and 100 Validation Images and 232 test Images.**

In [ ]:
'''This function makes pairs of directories of Image and its mask '''
def make_pair(img_lst,image_dir,mask_dir):
    pairs = []
    #print(image_dir+img_lst[0])
    for im in img_lst:
        pairs.append((image_dir + im, mask_dir + im.split('.')[0]+'_L.png'))
        
    return pairs

In [ ]:
'''Here we create lists of pairs of images and corresponding masks for both train and validation Images'''
train_pairs = make_pair(train_img_lst, "../input/camvid/CamVid/train/", 
                        "../input/camvid/CamVid/train_labels/")

val_pairs = make_pair(val_img_lst, "../input/camvid/CamVid/val/", 
                        "../input/camvid/CamVid/val_labels/")

test_pairs = make_pair(test_img_lst, "../input/camvid/CamVid/test/", 
                        "../input/camvid/CamVid/test_labels/")

test_pairs[0]

**From above output we see that every element of the lists train_pairs and test pairs is a pair of directories of image and its corresponding mask**

In [ ]:
'''We can simply plot and see the image and corresponding mask from above list of directories randomly'''
temp = choice(train_pairs)
img = img_to_array(load_img(temp[0]))
mask = img_to_array(load_img(temp[1]))
#mask_pil = np.asarray(Image.open(temp[1]))

plt.figure(figsize=(12,12))
plt.subplot(121)
plt.title("Image")
plt.imshow(img/255)
plt.subplot(122)
plt.title("Mask")
plt.imshow(mask/255)
#plt.subplot(123)
#plt.imshow(mask_pil)
plt.show()

**Here We get a clear feel about what task we have to carry out, i.e. what is Image Segmentation.**

**A CSV file of Class map is also provided , which gives us the mapping, we will load it and see how mapping works**

In [ ]:
class_map_df = pd.read_csv("../input/camvid/CamVid/class_dict.csv")
class_map_df.head()

**So, we can clearly see that class map is a mapping which gives us , object->class mapping as for example Animal belongs to class 0. And object -> RGB values mapping for example Animal which belongs to class zero should have R=64, G=128 and B=64 as its pixel values.**

**This class map csv file defines 32 different classes and their RGB values to be mapped. So, we convert it into List whose index will give Class and entry at that index gives RGB values of that class/index.**

In [ ]:
class_map = []
for index,item in class_map_df.iterrows():
    class_map.append(np.array([item['r'], item['g'], item['b']]))
    
print(len(class_map))
print(class_map[0])

In [ ]:
"""This function will be used later, to assert that mask should contains values that are class labels only.
   Like, our example has 32 classes , so predicted mask must contains values between 0 to 31. 
   So that it can be mapped to corresponding RGB."""
def assert_map_range(mask,class_map):
    mask = mask.astype("uint8")
    for j in range(img_size):
        for k in range(img_size):
            assert mask[j][k] in class_map , tuple(mask[j][k])

In [ ]:
'''This method will convert mask labels(to be trained) from RGB to a 2D image whic holds class labels of the pixels.'''
def form_2D_label(mask,class_map):
    mask = mask.astype("uint8")
    label = np.zeros(mask.shape[:2],dtype= np.uint8)
    
    for i, rgb in enumerate(class_map):
        label[(mask == rgb).all(axis=2)] = i
    
    return label

**So our above plotted mask Image will get converted into a 2D label with the class labels at corresponding pixels.**

In [ ]:
lab = form_2D_label(mask,class_map)
np.unique(lab,return_counts=True)

**Classes present in the particular mask and no. of pixels belonging to that class**

**Custom Data Generator**

**This class is a custom datagenerator, which takes pairs list and return images and 2D labeled masks. We will use this generator to feed our model.**

In [ ]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    
    def __init__(self, pair,class_map,  batch_size=16, dim=(224,224,3), shuffle=True):
        'Initialization'
        self.dim = dim
        self.pair = pair
        self.class_map = class_map
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.pair) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [k for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.pair))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        batch_imgs = list()
        batch_labels = list()

        # Generate data
        for i in list_IDs_temp:
            # Store sample
            img = load_img(self.pair[i][0] ,target_size=self.dim)
            img = img_to_array(img)/255.
            batch_imgs.append(img)

            label = load_img(self.pair[i][1],target_size=self.dim)
            label = img_to_array(label)
            #------ comment these two lines to see proper working of datagenerator in cell below----#
            label = form_2D_label(label,self.class_map)
            label = np.asarray(to_categorical(label , num_classes = 32))
            #------ But after that uncomment again them before training the model----------#
            #------ comment them to just run the cell below and again uncomment these two lines----#
            #print(label.shape)
            batch_labels.append(label)
        return np.array(batch_imgs) ,np.array(batch_labels)

**Creating objects of Class Datagenerator and trying to plot what it returns to validate its proper working.**

In [ ]:
img_size = 512
#class_map = class_palette()

train_generator1 = DataGenerator(train_pairs,class_map,batch_size=4, dim=(img_size,img_size,3) ,shuffle=True)
X,y = train_generator1.__getitem__(0)
print(X.shape, y.shape)


plt.figure(figsize=(12, 6))
print("Images")
for i in range(4):
    plt.subplot(2, 4, i+1)
    plt.imshow(X[i])
plt.show()

print("Masks")
plt.figure(figsize=(12, 6))
for i in range(4):
    plt.subplot(2, 4, i+1)
    plt.imshow(y[i])
plt.show()

**Go and uncomment those lines in Datagenerator class, __data_generation() method and run that cell again.**

In [ ]:
train_generator = DataGenerator(train_pairs,class_map,batch_size=4, dim=(img_size,img_size,3) ,shuffle=True)
val_generator = DataGenerator(val_pairs, class_map, batch_size=4, dim=(img_size,img_size,3) ,shuffle=True)
test_generator = DataGenerator(test_pairs, class_map, batch_size=4, dim=(img_size,img_size,3) ,shuffle=True)

**U-Net Model**

In [ ]:
def conv_block(tensor, nfilters, size=3, padding='same', initializer="he_normal"):
    x = Conv2D(filters=nfilters, kernel_size=(size, size), padding=padding, kernel_initializer=initializer)(tensor)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(filters=nfilters, kernel_size=(size, size), padding=padding, kernel_initializer=initializer)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x


def deconv_block(tensor, residual, nfilters, size=3, padding='same', strides=(2, 2)):
    y = Conv2DTranspose(nfilters, kernel_size=(size, size), strides=strides, padding=padding)(tensor)
    y = concatenate([y, residual], axis=3)
    y = conv_block(y, nfilters)
    return y
def Unet(h, w, filters, num_classes = 32):
# down
    input_layer = Input(shape=(h, w, 3), name='image_input')
    conv1 = conv_block(input_layer, nfilters=filters)
    conv1_out = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = conv_block(conv1_out, nfilters=filters*2)
    conv2_out = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = conv_block(conv2_out, nfilters=filters*4)
    conv3_out = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = conv_block(conv3_out, nfilters=filters*8)
    conv4_out = MaxPooling2D(pool_size=(2, 2))(conv4)
    conv4_out = Dropout(0.5)(conv4_out)
    conv5 = conv_block(conv4_out, nfilters=filters*16)
    conv5 = Dropout(0.5)(conv5)
# up
    deconv6 = deconv_block(conv5, residual=conv4, nfilters=filters*8)
    deconv6 = Dropout(0.5)(deconv6)
    deconv7 = deconv_block(deconv6, residual=conv3, nfilters=filters*4)
    deconv7 = Dropout(0.5)(deconv7) 
    deconv8 = deconv_block(deconv7, residual=conv2, nfilters=filters*2)
    deconv9 = deconv_block(deconv8, residual=conv1, nfilters=filters)
    output_layer = Conv2D(filters=num_classes, kernel_size=(1, 1), activation='softmax')(deconv9)

    model = Model(inputs=input_layer, outputs=output_layer, name='Unet')
    return model

In [ ]:
model = Unet(img_size , img_size , 64)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy' ,metrics=['accuracy'])

In [ ]:
mc = ModelCheckpoint(mode='max', filepath='top-weights.h5', monitor='val_acc',save_best_only='True', verbose=1)
es = EarlyStopping(monitor='val_acc', patience=10, verbose=0)

In [ ]:
train_steps = train_generator.__len__()
val_steps = val_generator.__len__()

print(train_steps, val_steps)

**I trained the model and saved it and loaded again to evaluate**

In [ ]:
results = model.fit_generator(train_generator , steps_per_epoch=train_steps ,epochs=30,
                              validation_data=val_generator,validation_steps=val_steps,callbacks=[mc,es],
                             verbose = 1)

In [ ]:
model.save('./camvid_unet_model.h5')

**Load the saved model to evaluate it**

In [ ]:
trained_model = keras.models.load_model("../input/unet-model/camvid_unet_model.h5")

In [ ]:
trained_model.evaluate_generator(test_generator, verbose=1)

**Test Accuracy is near 84-85%. And loss is 0.57.**

**Let's compare the model predictions with original masks**

In [ ]:
x_test, y_test = test_generator.__getitem__(2)
print(x_test.shape, y_test.shape)

In [ ]:
y_pred = trained_model.predict(x_test, verbose = 1, batch_size = 4)
y_pred.shape

In [ ]:
'''This converts predicted map to RGB labels'''
def map_this(y_pred,class_map):
    y_pred_rgb = np.zeros((y_pred.shape[0],y_pred.shape[1],y_pred.shape[2],3))
    for i in range(y_pred.shape[0]):
        image = np.zeros((y_pred.shape[1],y_pred.shape[2],3))
        for j in range(y_pred.shape[1]):
            for k in range(y_pred.shape[2]):
                image[j,k,:] = class_map[y_pred[i][j][k]]
        y_pred_rgb[i] = image
    return y_pred_rgb

In [ ]:
"""This will plot original image, original mask and predicted mask"""
def plot_result(img , title):
    plt.figure(figsize=(12, 6))
    plt.title(title)
    for i in range(4):
        #print(pred[i].shape)
        plt.subplot(2, 4, i+1)
        plt.imshow(img[i])
    plt.show()

In [ ]:
pred = np.argmax(y_pred, axis=3)
y_pred_rgb = map_this(pred,class_map)
test = np.argmax(y_test, axis=3)
y_test_rgb = map_this(test,class_map)

In [ ]:
plot_result(x_test,"Test Images")

In [ ]:
plot_result(y_test_rgb,"Original Masks")

In [ ]:
plot_result(y_pred_rgb,"Predicted mask")

**Thanks and please upvote if you get to learn something new.**